# Carbon Intensity on the Grid

This notebook can be used to access real-time carbon intensity analysis of electrical grids worldwide. It demonstrates how to leverage the [Electricity Maps API](https://www.electricitymaps.com/free-tier-api) to analyze how carbon intensity varies across different regions and to make data-driven decisions about where and when to deploy machine learning workloads with a lower environmental impact.
 
### Overview
- **Real-time Carbon Monitoring**: access live carbon intensity data from global electrical grids
- **Geographic Comparison**: compare carbon footprints across different regions and time zones
- **Energy Source Analysis**: understand the renewable vs. fossil fuel mix powering different locations
- **ML Training Optimization**: identify optimal locations for energy-intensive AI model training

## Setup Environment and Load API 
- Load environmemnt variables containing Electricity Maps API key. 
- Use a helper function to load the Electricity Maps API credentials securely from environment variables.


In [ ]:
from helper import load_env
load_env()

## Location Coordinates
Define the geographic coordinates for Los Angeles, California. These coordinates will be used to query the specific electrical grid zone `US-CAL-LDWP`.

> Coordinates can be obtained through [Google Maps](https://www.google.com/maps).

In [ ]:
# Create a dictionary with the coordinates
coordinates = {
    "lat":34.00906474557528,
    "lon": -118.4984580927553
}

## Electricity Map's Live Carbon Intensity Endpoint

Construct the REST API endpoint URL for retrieving live carbon intensity data. The endpoint uses latitude/longitude parameters to automatically determine the appropriate electrical grid zone.

**API Endpoint**: `/v3/carbon-intensity/latest` - Returns the most recent carbon intensity measurement for the specified location.

In [ ]:
# Build the url
url= f"https://api.electricitymap.org/v3/carbon-intensity/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"

# Print the endpoint
print("Endpoint: " + str(url))

Endpoint: https://api.electricitymap.org/v3/carbon-intensity/latest?lat=34.00906474557528&lon=-118.4984580927553


### Import Libraries

In [4]:
import requests
import helper

### Carbon Intensity API Request

Execute the HTTP GET request to the Electricity Maps API. 
>**Authentication**: Uses bearer token authentication via the `auth-token` header field. 

In [5]:
request = requests.get(
    url,
    headers={"auth-token": helper.load_emaps_api_key()})

### Response 

Raw response format returned by the API. The data comes as `bytes`, which is more compact for network transmission but requires parsing for human readability.

`<class 'bytes'>` - Raw binary data format that needs JSON parsing.

In [6]:
# This byte format is more compact
request.content
type(request.content)

bytes

## Carbon Intensity Data Parsing

`import json` to parse the raw API responseinto a structured Python dictionary containing carbon intensity metrics and metadata.

**Key Fields**:
- `carbonIntensity`: 700 gCO2eq/kWh - The carbon emissions per kilowatt-hour
- `zone`: 'US-CAL-LDWP' - Los Angeles Department of Water and Power grid zone
- `datetime`: Timestamp of the measurement
- `isEstimated`: Boolean indicating if the value is estimated vs. measured
- `estimationMethod`: Algorithm used for estimation when real-time data unavailable

In [7]:
import json

In [8]:
json.loads(request.content)

{'zone': 'US-CAL-LDWP',
 'carbonIntensity': 700,
 'datetime': '2024-07-22T15:00:00.000Z',
 'updatedAt': '2024-07-22T14:49:34.787Z',
 'createdAt': '2024-07-19T15:46:30.525Z',
 'emissionFactorType': 'lifecycle',
 'isEstimated': True,
 'estimationMethod': 'TIME_SLICER_AVERAGE'}

## Live Power Breakdown Endpoint

Construct the API endpoint for retrieving detailed power generation breakdown by source type (solar, wind, gas, coal, etc.).

**API Endpoint**: `/v3/power-breakdown/latest` - Returns the energy mix composition for the specified grid zone.

In [9]:
# Build the url
url = f"https://api.electricitymap.org/v3/power-breakdown/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"

In [10]:
print(url)

https://api.electricitymap.org/v3/power-breakdown/latest?lat=34.00906474557528&lon=-118.4984580927553


#### Execute the HTTP request to retrieve detailed power generation breakdown data from the same grid zone.

In [11]:
request = requests.get(
    url,
    headers={"auth-token": helper.load_emaps_api_key()})

## Power Breakdown Data Analysis

Parse and display the comprehensive power generation breakdown, revealing the energy source composition of the electrical grid.

**Output Insights**:
- **100% Unknown Sources**: All 219 MW consumption is categorized as "unknown"
- **0% Renewable**: No renewable energy sources detected
- **Grid Limitation**: This specific zone (US-CAL-LDWP) has limited data granularity

In [12]:
power_breakdown = json.loads(request.content)

# Print the content
power_breakdown

{'zone': 'US-CAL-LDWP',
 'datetime': '2024-07-22T15:00:00.000Z',
 'updatedAt': '2024-07-22T14:49:34.787Z',
 'createdAt': '2024-07-19T15:46:30.525Z',
 'powerConsumptionBreakdown': {'nuclear': 0,
  'geothermal': 0,
  'biomass': 0,
  'coal': 0,
  'wind': 0,
  'solar': 0,
  'hydro': 0,
  'gas': 0,
  'oil': 0,
  'unknown': 219,
  'hydro discharge': 0,
  'battery discharge': 0},
 'powerProductionBreakdown': {'nuclear': None,
  'geothermal': None,
  'biomass': None,
  'coal': None,
  'wind': None,
  'solar': None,
  'hydro': None,
  'gas': None,
  'oil': None,
  'unknown': 219,
  'hydro discharge': None,
  'battery discharge': None},
 'powerImportBreakdown': {},
 'powerExportBreakdown': {},
 'fossilFreePercentage': 0,
 'renewablePercentage': 0,
 'powerConsumptionTotal': 219,
 'powerProductionTotal': 219,
 'powerImportTotal': None,
 'powerExportTotal': None,
 'isEstimated': True,
 'estimationMethod': 'TIME_SLICER_AVERAGE'}

## Renewable Energy Metrics

>**Renewable Percentage**: 0% indicates no renewable energy in the current mix

In [13]:
power_breakdown['renewablePercentage']

0

> **Fossil Free Percentage**: 0% indicates complete reliance on fossil fuels or unknown sources

In [14]:
power_breakdown['fossilFreePercentage']

0

>**Consumption Breakdown**: Shows 219 MegaWatts total consumption, all from unknown sources

In [15]:
# Power Consumption Breakdown in MegaWatts
power_breakdown['powerConsumptionBreakdown']

{'nuclear': 0,
 'geothermal': 0,
 'biomass': 0,
 'coal': 0,
 'wind': 0,
 'solar': 0,
 'hydro': 0,
 'gas': 0,
 'oil': 0,
 'unknown': 219,
 'hydro discharge': 0,
 'battery discharge': 0}

## Consumption Percentage Calculation

Calculate the percentage contribution of each energy source to total consumption, providing a normalized view of the energy mix.

**Result**: 100% unknown sources, indicating data limitations for this specific grid zone.

In [16]:
import numpy as np

In [17]:
total_consumption = power_breakdown['powerConsumptionTotal']

In [18]:
total_consumption

219

## Reusable Power Statistics Function

Create a reusable function that combines both carbon intensity and power breakdown queries into a single, convenient interface.

**Function Features**:
- **Dual API Calls**: Retrieves both carbon intensity and power breakdown data
- **Data Processing**: Calculates percentage breakdowns automatically
- **Clean Interface**: Returns structured data for easy analysis
- **Error Handling**: Built-in API key management

In [19]:
consumption_percent = {
    k: np.round((v/total_consumption) * 100)
    for k,v
    in power_breakdown['powerConsumptionBreakdown'].items()}
consumption_percent

{'nuclear': 0.0,
 'geothermal': 0.0,
 'biomass': 0.0,
 'coal': 0.0,
 'wind': 0.0,
 'solar': 0.0,
 'hydro': 0.0,
 'gas': 0.0,
 'oil': 0.0,
 'unknown': 100.0,
 'hydro discharge': 0.0,
 'battery discharge': 0.0}

## Taiwan Case Study

Demonstrate the `power_stats` helper function with Taiwan's electrical grid as a comparative case study.

#### Grid Analysis:
- **Carbon Intensity**: 570 gCO2eq/kWh (lower than Los Angeles)
- **Energy Mix**: 50% natural gas, 34% coal, 5% nuclear
- **Renewable Share**: 5% (wind + hydro + solar)
- **Total Consumption**: ~34 GW across all sources

**Implications**:
- Taiwan shows significantly more detailed data than the LA zone
- Heavy reliance on fossil fuels (84% gas + coal combined)
- Limited but measurable renewable energy contribution
- Nuclear power provides 5% of the energy mix

In [20]:
import helper, requests, json, numpy as np
def power_stats(lat,lon, api_key=helper.load_emaps_api_key()):
    coordinates = { "lat": lat, "lon": lon }

    url_intensity = f"https://api.electricitymap.org/v3/carbon-intensity/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"
    request_intensity = requests.get(url_intensity, headers={"auth-token": api_key})
    intensity = json.loads(request_intensity.content)

    url_breakdown = f"https://api.electricitymap.org/v3/power-breakdown/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"
    request_breakdown = requests.get(url_breakdown, headers={"auth-token": api_key})
    breakdown = json.loads(request_breakdown.content)

    breakdown_abridged = {
        'renewablePercentage': breakdown['renewablePercentage'],
        'fossilFreePercentage': breakdown['fossilFreePercentage'],
        'powerConsumptionBreakdown': breakdown['powerConsumptionBreakdown'],
        'consumption_percent': {
            k: np.round((v/breakdown['powerConsumptionTotal']) * 100) 
            for k, v 
            in breakdown['powerConsumptionBreakdown'].items()
        },
    }
    
    return intensity, breakdown_abridged

In [21]:
# Coordinates from a landmark in Taiwan
intensity, breakdown = power_stats(
    lat=25.0356575108668,
    lon=121.52010809479746)

In [22]:
intensity

{'zone': 'TW',
 'carbonIntensity': 570,
 'datetime': '2024-07-22T15:00:00.000Z',
 'updatedAt': '2024-07-22T14:49:34.787Z',
 'createdAt': '2024-07-19T15:46:17.152Z',
 'emissionFactorType': 'lifecycle',
 'isEstimated': True,
 'estimationMethod': 'TIME_SLICER_AVERAGE'}

In [23]:
breakdown

{'renewablePercentage': 5,
 'fossilFreePercentage': 9,
 'powerConsumptionBreakdown': {'nuclear': 1637,
  'geothermal': 2,
  'biomass': 20,
  'coal': 11426,
  'wind': 401,
  'solar': 0,
  'hydro': 1154,
  'gas': 16907,
  'oil': 933,
  'unknown': 1386,
  'hydro discharge': 0,
  'battery discharge': 0},
 'consumption_percent': {'nuclear': 5.0,
  'geothermal': 0.0,
  'biomass': 0.0,
  'coal': 34.0,
  'wind': 1.0,
  'solar': 0.0,
  'hydro': 3.0,
  'gas': 50.0,
  'oil': 3.0,
  'unknown': 4.0,
  'hydro discharge': 0.0,
  'battery discharge': 0.0}}